In [8]:
%matplotlib inline

import requests
import numpy as np
import pandas as pd

## Preface

This is just a reference - the referenced file is not in the repo.
Just a walkthrough on what happens in the original data since there are some unwanted and missing data all throughout.

Date is parsed to retain YYYY-MM-DD format.

In [ ]:
#pd.read_csv('https://bit.ly/3dmark_xz', compression='xz')
data = pd.read_csv("data/combined.csv.xz", parse_dates=["date"]) #file isn't in this directory
data.benchmark_version = data.benchmark_version.astype("str")
data.info()

The columns below are dropped due to having seriously lacking data (around 50% of the dataset). There are non-average alternatives for these measurements except for the temperature. Imputing the temperature would be impossible.

In [94]:
data.drop([
    "cmbnd_score", 
    "cmbnd_fps", 
    "gpu_avg_temp",
    "gpu_avg_clock_freq",
    "gpu_avg_memclock_freq"], axis=1, inplace=True)

Unfinished benchmark runs return 0 as score - removing those results are necessary as they could skew everything with a long negative tail.

In [95]:
data = data.loc[data.score != 0]

RAM clocks were saved erroneously due to some results not having them at all.

For the meantime, they will be replaced with Numpy NaNs to be removed later after conversion.

In [96]:
data.ram_clock_freq = data.ram_clock_freq.str.replace('[^0-9\s]+', '')
data.ram_clock_freq.replace('', np.nan, inplace=True)
data.gpu_score = data.gpu_score.str.replace('[^0-9\s\r\n]+', '')
data.gpu_score.replace('', np.nan, inplace=True)

In [97]:
# typecasting ram_clock_freq
data.ram_clock_freq = pd.to_numeric(data.ram_clock_freq)

# typecasting gpu_score
data.gpu_score = pd.to_numeric(data.gpu_score)

After converting RAM clocks and GPU scores to numeric, let's have a peek at results retained.

In [98]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1190273 entries, 0 to 1246200
Data columns (total 30 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1190273 non-null  int64         
 1   benchmark           1190273 non-null  object        
 2   benchmark_version   1190273 non-null  object        
 3   cpu_clock_freq      1190273 non-null  int64         
 4   cpu_cores           1190273 non-null  int64         
 5   cpu_litho           1190273 non-null  int64         
 6   cpu_make            1190273 non-null  object        
 7   cpu_score           1190273 non-null  int64         
 8   cpu_sku             1190273 non-null  object        
 9   cpu_socket          1190194 non-null  object        
 10  cpu_tdp             1190273 non-null  int64         
 11  cpu_fps             1190273 non-null  float64       
 12  date                1190273 non-null  datetime64[ns]
 13  gpu         

In [99]:
data.isnull().mean()*100

id                    0.000000
benchmark             0.000000
benchmark_version     0.000000
cpu_clock_freq        0.000000
cpu_cores             0.000000
cpu_litho             0.000000
cpu_make              0.000000
cpu_score             0.000000
cpu_sku               0.000000
cpu_socket            0.006637
cpu_tdp               0.000000
cpu_fps               0.000000
date                  0.000000
gpu                   0.000000
gpu_clock_freq        0.000000
gpu_count             0.000000
gpu_driver_status     0.000000
gpu_driver_version    0.025960
gpu_memclock_freq     0.000000
gpu_score             1.376491
gpu_vendor            0.000000
gpu_vram              0.000000
gt1                   0.000000
gt2                   0.000000
motherboard           0.000000
os                    0.000000
ram                   0.000000
ram_clock_freq        2.176728
score                 0.000000
storage               0.000000
dtype: float64

Overall, there are 2.177% results that will be pruned from the dataset. Imputing came into mind but the loss is negligible.

In [100]:
data.dropna(inplace=True)
pd.set_option('display.max_columns', None)
data.head()

,id,benchmark,benchmark_version,cpu_clock_freq,cpu_cores,cpu_litho,cpu_make,cpu_score,cpu_sku,cpu_socket,cpu_tdp,cpu_fps,date,gpu,gpu_clock_freq,gpu_count,gpu_driver_status,gpu_driver_version,gpu_memclock_freq,gpu_score,gpu_vendor,gpu_vram,gt1,gt2,motherboard,os,ram,ram_clock_freq,score,storage
16385,9127260,Time Spy,1.0,4201,4,22,Intel,2968,Core i5-4690K Processor,LGA1150,88,9.97,2019-11-04,NVIDIA GeForce GTX 1660 Ti,1950,1,Approved,25.21.14.1967,1500,6436.0,"Giga-Byte Technology Co., Ltd.",6144,41.67,37.12,MSI Z97-S02 (MS-7821),64-bit Windows 10 (10.0.18362),8192,1332.0,5476,250 GB Samsung SSD 840 EVO 250GB
16386,11409122,Time Spy,1.0,3992,6,14,Intel,5246,Core i5-9400F Processor,FCLGA1151,65,17.63,2020-04-10,NVIDIA GeForce GTX 1660 Ti,1950,1,Approved,26.21.14.4250,1500,6324.0,NVidia Corporation,6144,40.73,36.65,ASUSTeK COMPUTER INC. EX-B360M-V3,64-bit Windows 10 (10.0.17763),16384,2658.0,6134,500 GB Samsung SSD 860 EVO 500GB
16387,13902739,Time Spy,1.2,4167,6,7,AMD,7300,Ryzen 5 3600,AM4,65,24.53,2020-09-16,NVIDIA GeForce GTX 1660 Ti,1980,1,Approved,27.21.14.5206,1534,6437.0,NVidia Corporation,6144,41.72,37.09,"Gigabyte Technology Co., Ltd. B450M GAMING",64-bit Windows 10 (10.0.19041),16384,3592.0,6553,240 GB ADATA SU650
16388,11042706,Time Spy,1.0,4800,8,14,Intel,10113,Core i9-9900K Processor,FCLGA1151,95,33.98,2020-03-19,NVIDIA GeForce GTX 1660 Ti,1875,1,Approved,26.21.14.4259,1500,6089.0,ZOTAC,6144,39.10,35.38,ASRock Z390 Phantom Gaming 6,64-bit Windows 10 (10.0.18362),32768,2400.0,6475,"1,024 GB SPCC M.2 PCIe SSD"
16389,10064806,Time Spy,1.0,4459,6,12,AMD,6084,Ryzen 5 2600X,AM4,95,20.44,2020-01-07,NVIDIA GeForce GTX 1660 Ti,2010,1,Approved,26.21.14.3086,1500,6234.0,"Micro-Star International Co ., Ltd.",6144,40.11,36.16,ASRock B450M Pro4,64-bit Windows 10 (10.0.18363),16384,2992.0,6211,240 GB KINGSTON SA400M8240G


For further cleanup, some variables had 0 results. Selecting those that have non-zero values for those columns are necessary to preserve the data.

In [101]:
data = data.loc[~(
    (data['cpu_fps'] == 0) | 
    (data['gt1'] == 0) | 
    (data['gt2'] == 0) | 
    (data['ram'] == 0) |
    (data['cpu_litho'] == 0) |
    (data['ram_clock_freq'] == 0))]
data = data.loc[data.cpu_clock_freq < 8000]
data = data.loc[data.ram_clock_freq < 5000.0]

In [102]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1138545 entries, 16385 to 1246200
Data columns (total 30 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1138545 non-null  int64         
 1   benchmark           1138545 non-null  object        
 2   benchmark_version   1138545 non-null  object        
 3   cpu_clock_freq      1138545 non-null  int64         
 4   cpu_cores           1138545 non-null  int64         
 5   cpu_litho           1138545 non-null  int64         
 6   cpu_make            1138545 non-null  object        
 7   cpu_score           1138545 non-null  int64         
 8   cpu_sku             1138545 non-null  object        
 9   cpu_socket          1138545 non-null  object        
 10  cpu_tdp             1138545 non-null  int64         
 11  cpu_fps             1138545 non-null  float64       
 12  date                1138545 non-null  datetime64[ns]
 13  gpu     

The file is still huge if saved as a CSV. To lessen total project size, I will be saving this new dataframe with xz compression.

In [103]:
compression_settings = dict(method="xz", archive_name="test.csv")
data.to_csv("3dmarktimespy.csv.xz", index=False, compression=compression_settings)

In [104]:
ndata = pd.read_csv("3dmarktimespy.csv.xz") #not the final name
ndata.head()

,id,benchmark,benchmark_version,cpu_clock_freq,cpu_cores,cpu_litho,cpu_make,cpu_score,cpu_sku,cpu_socket,cpu_tdp,cpu_fps,date,gpu,gpu_clock_freq,gpu_count,gpu_driver_status,gpu_driver_version,gpu_memclock_freq,gpu_score,gpu_vendor,gpu_vram,gt1,gt2,motherboard,os,ram,ram_clock_freq,score,storage
0,9127260,Time Spy,1.0,4201,4,22,Intel,2968,Core i5-4690K Processor,LGA1150,88,9.97,2019-11-04,NVIDIA GeForce GTX 1660 Ti,1950,1,Approved,25.21.14.1967,1500,6436.0,"Giga-Byte Technology Co., Ltd.",6144,41.67,37.12,MSI Z97-S02 (MS-7821),64-bit Windows 10 (10.0.18362),8192,1332.0,5476,250 GB Samsung SSD 840 EVO 250GB
1,11409122,Time Spy,1.0,3992,6,14,Intel,5246,Core i5-9400F Processor,FCLGA1151,65,17.63,2020-04-10,NVIDIA GeForce GTX 1660 Ti,1950,1,Approved,26.21.14.4250,1500,6324.0,NVidia Corporation,6144,40.73,36.65,ASUSTeK COMPUTER INC. EX-B360M-V3,64-bit Windows 10 (10.0.17763),16384,2658.0,6134,500 GB Samsung SSD 860 EVO 500GB
2,13902739,Time Spy,1.2,4167,6,7,AMD,7300,Ryzen 5 3600,AM4,65,24.53,2020-09-16,NVIDIA GeForce GTX 1660 Ti,1980,1,Approved,27.21.14.5206,1534,6437.0,NVidia Corporation,6144,41.72,37.09,"Gigabyte Technology Co., Ltd. B450M GAMING",64-bit Windows 10 (10.0.19041),16384,3592.0,6553,240 GB ADATA SU650
3,11042706,Time Spy,1.0,4800,8,14,Intel,10113,Core i9-9900K Processor,FCLGA1151,95,33.98,2020-03-19,NVIDIA GeForce GTX 1660 Ti,1875,1,Approved,26.21.14.4259,1500,6089.0,ZOTAC,6144,39.10,35.38,ASRock Z390 Phantom Gaming 6,64-bit Windows 10 (10.0.18362),32768,2400.0,6475,"1,024 GB SPCC M.2 PCIe SSD"
4,10064806,Time Spy,1.0,4459,6,12,AMD,6084,Ryzen 5 2600X,AM4,95,20.44,2020-01-07,NVIDIA GeForce GTX 1660 Ti,2010,1,Approved,26.21.14.3086,1500,6234.0,"Micro-Star International Co ., Ltd.",6144,40.11,36.16,ASRock B450M Pro4,64-bit Windows 10 (10.0.18363),16384,2992.0,6211,240 GB KINGSTON SA400M8240G


In [105]:
ndata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138545 entries, 0 to 1138544
Data columns (total 30 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1138545 non-null  int64  
 1   benchmark           1138545 non-null  object 
 2   benchmark_version   1138545 non-null  float64
 3   cpu_clock_freq      1138545 non-null  int64  
 4   cpu_cores           1138545 non-null  int64  
 5   cpu_litho           1138545 non-null  int64  
 6   cpu_make            1138545 non-null  object 
 7   cpu_score           1138545 non-null  int64  
 8   cpu_sku             1138545 non-null  object 
 9   cpu_socket          1138545 non-null  object 
 10  cpu_tdp             1138545 non-null  int64  
 11  cpu_fps             1138545 non-null  float64
 12  date                1138545 non-null  object 
 13  gpu                 1138545 non-null  object 
 14  gpu_clock_freq      1138545 non-null  int64  
 15  gpu_count      

In [ ]:
# dtype mapping for later use in project
combined_dtypes = {
    "id" : np.uint32,
    "benchmark_version" : np.float16,
    "cpu_clock_freq" : np.float16,
    "cpu_cores" : np.uint8,
    "cpu_litho" : np.uint8,
    "cpu_score" : np.float16,
    "cpu_tdp" : np.uint8,
    "cpu_fps" : np.float16,
    "gpu_clock_freq" : np.float16,
    "gpu_count" : np.uint8,
    "gpu_memclock_freq" : np.float16,
    "gpu_score" : np.uint32,
    "gpu_vram" : np.uint16,
    "gt1" : np.float16,
    "gt2" : np.float16,
    "ram" : np.uint16,
    "score" : np.uint16,
}

Compression was a success! The data integrity is retained with no loss in information. This can now be used for projects.

Do note that you should copy the dtype mapping above to lessen size and improve runtimes.